In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
import numpy as np
import tensorflow as tf
import os, sys, time
os.environ["CUDA_VISIBLE_DEVICES"]=""
import utils, optimization, metrics, plot

# Parameters

In [ ]:
spix = 256
path = '../data/size{}_splits1000_n500x3/'.format(spix)
input_pattern = 'Box_70*snapshot_050'
file_ext = '.dat'
k = 10

# Data handling

Load the data

In [ ]:
queue = []
for file in os.listdir(path):
    if file.endswith(file_ext) and (np.all([x in file for x in input_pattern.split("*")])):
        queue.append(os.path.join(path, file))
nsamples = len(queue)
print('They are {} "{}" files.'.format(nsamples, file_ext))      

In [ ]:
nsamples = 1000
input_img = np.vstack(map(lambda i:np.fromfile(queue[i], dtype=float32), range(nsamples)))
input_img.resize([nsamples,spix,spix])


In [ ]:
input_img = utils.forward_map(input_img,k)

# input_img = np.log(input_img+0.001)

# p = np.random.permutation(nsamples)
# input_img = input_img[p,]

Let us make small patches of 64 :-)

# A) Load the model

In [ ]:
import pickle
from model import *
from gan import *

try:
    del(params)
    del(obj)
except:
    pass

checkpoints = None


# pathgan = '../../saved_result/WGAN64_2018_1_14_19_28checkpoints/'
# checkpoints = '12000'
# ns = 64


# pathgan = '../../saved_result/WGAN128_2018_1_14_21_8checkpoints/'
# checkpoints = '12600'
# ns = 128

pathgan = '../../saved_result/WGAN256_2018_1_13_14_25checkpoints/'
checkpoints = '15000'
ns = 256


cut = spix//ns
img_small = np.zeros([nsamples*cut*cut, ns, ns])
for i in range(cut):
    for j in range(cut):
        l = j + i*cut 
        img_small[l*nsamples:(l+1)*nsamples,:,:] = input_img[:,i*ns:(i+1)*ns,j*ns:(j+1)*ns]


with open(pathgan+'params.pkl', 'rb') as f:
    params = pickle.load(f)
obj = GAN(params)


# Generate some samples

In [ ]:
N = 500


if checkpoints is None:
    gen_sample, gen_sample_raw = obj.generate(N=N)
else:
    file_name = pathgan+obj.model_name+'-'+checkpoints
    
    gen_sample, gen_sample_raw = obj.generate(N=N, file_name=file_name)

gen_sample = np.squeeze(gen_sample)
gen_sample_raw = np.squeeze(gen_sample_raw)
real_sample_raw = utils.backward_map(img_small[:N],k=k)

Print a few samples

In [ ]:
plt.figure(figsize=(15,15))
utils.draw_images(gen_sample,nx=10,ny=10,px=ns,py=ns)

# Evaluate the model

In [ ]:
psd_gen, x = metrics.power_spectrum_batch_phys(X1=gen_sample_raw)
psd_real, x = metrics.power_spectrum_batch_phys(X1=real_sample_raw)


def psd_metric(gen_sample_raw, real_sample_raw):
    psd_gen, _ = metrics.power_spectrum_batch_phys(X1=gen_sample_raw)
    psd_real, _ = metrics.power_spectrum_batch_phys(X1=real_sample_raw)
    psd_gen = np.mean(psd_gen, axis=0)    
    psd_real = np.mean(psd_real, axis=0)
    e = psd_real - psd_gen
    l2 = np.mean(e*e)
    loge = 10*(log10(psd_real+1e-5) - log10(psd_gen+1e-5))
    logel2 = np.mean(loge*loge)
    return l2, logel2
l2, logel2 = psd_metric(gen_sample_raw, real_sample_raw)


In [ ]:
print('Log PSD loss: {}\n L2 PSD loss: {}'.format(logel2, l2))

In [ ]:
fig = plt.Figure()
ax = plt.gca()
ax.set_xscale("log")
ax.set_yscale("log")
linestyle = {"linewidth": 1, "markeredgewidth": 0, "markersize": 3, "marker": "o", "linestyle": "-"}

plot.plot_with_shade(ax, x, psd_gen, color='r', label="Fake $\mathcal{F}(X))^2$", **linestyle)
plot.plot_with_shade(ax, x, psd_real, color='b', label="Real $\mathcal{F}(X))^2$", **linestyle)
ax.set_ylim(bottom=0.1)
ax.title.set_text("2D Power Spectrum\n")
ax.title.set_fontsize(11)
ax.tick_params(axis='both', which='major', labelsize=10)
_ = ax.legend()


In [ ]:

sigma_smooth = [0,0.5,1]
plot.plot_images_psd(real_sample_raw,'' , sigma_smooth = sigma_smooth)
plot.plot_images_psd(gen_sample_raw,'' , sigma_smooth = sigma_smooth)